# Question 1

We wish to train a machine learning algorithm on an array of floating point numbers in the interval [0.0, 1.0). The data is horribly unbalanced (not evenly distributed) and we wish to filter the dataset to obtain a subset containing an equal number of values from each interval [0, 0.2), [0.2, 0.4), ... [0.8, 1.0), throwing away as little data as possible.

1. Write a program which reads comma-separated floating point numbers in a single line from stdin, and prints the filtered data to stdout in the same format

**Overview**

How should we balance the data? If our goal is only to obtain a dataset with the same number of values in each bucket, then we can do several things

1. As the data comes in, we bucket the data, keeping track of how many elements are in each bucket. Then, at output time, we output that many elements from each bucket.
 - this does *throw away as little data as possible*

What does it do when there are no values in one interval [0.8, 1.0)?
* Return nothing from all the intervals?

**Tests**

* Doesn't throw away data: if the number of elements in each bucket is the same to begin with



In [1]:
def cut(numbers, disp=False):
    """Filter the input to create a balanced data
    
    Params:
    numbers - array of floats in [0.0, 1.0)
    """
    intervals = [0.2, 0.4, 0.6, 0.8, 1.0]
    buckets = [[] for i in intervals]
    for num in numbers:
        # find the right bucket
        for i, right in enumerate(intervals):
            if num < right:
                buckets[i].append(num)
                break
    
    if disp:
        print('Buckets:\n', buckets)
        
    # find out how many values to keep
    k = min([len(b) for b in buckets])
    
    # take k values from each bucket
    buckets = [b[:k] for b in buckets]
    data = []
    for b in buckets:
        data += b
        
    return data


Let's test that `cut` works

In [2]:
# TEST: drops unbalanced elements
inputs = [0,0.1,0.3,0.4,0.5,0.7,0.9, 1.0]
expected = [0, 0.3, 0.4, 0.7, 0.9]
assert cut(inputs, True) == expected

# TEST: throws away as little as possible
inputs = [0, 0.1, 0.2, 0.3, 0.4, 0.4, 0.6, 0.6, 0.8, 0.8, 1.0, 1.0]
expected = [0, 0.1, 0.2, 0.3, 0.4, 0.4, 0.6, 0.6, 0.8, 0.8]
assert cut(inputs, True) == expected

Buckets:
 [[0, 0.1], [0.3], [0.4, 0.5], [0.7], [0.9]]
Buckets:
 [[0, 0.1], [0.2, 0.3], [0.4, 0.4], [0.6, 0.6], [0.8, 0.8]]


In [3]:
!echo 0, 0.1, 0.2, 0.3, 0.4, 0.4, 0.6, 0.6, 0.8, 0.8, 1.0, 1.0 | python filter.py


0.0,0.1,0.2,0.3,0.4,0.4,0.6,0.6,0.8,0.8


Of course, we'll need some end-to-end tests to make sure that the program output is what we expect... such as checking the output of the bash scripts (**todo**...)

# Question 2

Suppose you are asked to write a simple game involving a spaceship and an asteroid. The spaceship and an asteroid have a position and orientation in a 2D plane. The aim of the game is for the spaceship to fire a missile at the asteroid and destroy it.

1. Describe or draw a software design for this game.
    * this may include a system diagram, flow chart, class diagram, etc.
    * State any assumptions you make.
    
    
Writing this game involves thinking about three parts:

* The engine, which we'll call `Game` - Responsible for operating the core logic of the game, such as moving pieces. Advanced engines will have renderers (which we simulate here with a `render` method), as well as code for handling the interaction between various game objects

* The front-end - Omitted. This will usually be the part that handles user input (like a webpage, or a C++ UI), which makes calls to the engine and renders the result. Here, our input is nothing, and our output is the console!

* The game objects - How do we represent the pieces involved? In this doc, we'll make an object which encapsulates the idea of a 2D point which has its own velocity and orientation.
    * `Asteroid` - Moves in a straight line, spawns randomly
    * `Missile` - Moves in a straight line. Has a max range. When it hits an asteroid, they both disappear
    * `Spaceship` - Can accelerate in 2D, change its orientation, and fire a missile

## Design
#### Game

The `Game` is responsible for handling user input and moving the objects in the game

* `init()` - inits a spaceship and asteroid.
* `update()` - moves everything, checks for collisions
* `user_input(c)` - tells the spaceship to turn and accelerate. Also can fire a missile

**Assumptions**
- Collision checking - since the number of objects is small, no dedicated techniques (like quadtrees, or sorting) to reduce teh load
- space is infinite
- however, `Spaceship` will have a max speed
- the thing that calls `user_input` will be defined by the user - this can be a webpage, another program, etc

#### Game objects

All three things can `move()`. In addition, `Spaceship` can `turn(x)` and `accelerate(x)`.

**Assumptions**
- Based on the simple 1-asteroid scenario, I make every game object keep track of its own state! If we had a billion asteroids and one ship, we might look into structuring this storage differently (say, with a vector of all the positions)
- `move()` is thread-safe. When we `turn()` the spaceship, we need to make sure it doesn't interfere with the game logic of moving it!

#### Scope

we're stripping out a lot of functionality from the `Game` and spaceship classes. Notably:

* A renderer
* Loading screens
* Scores
* Multiplayer, remote, etc


2. Outline the code for the spaceship, asteroid, and missile.
    * Show any data structures and stubs for methods and/or functions that show the implementation of your design in part (a).
    * State any assumptions you make.
    * It is not necessary to make a functional game (in particular, do not worry about graphics or a game engine).

In [5]:
import numpy as np

class Gobject:
    def __init__(self, position, orientation, velocity=0.0, name="default"):
        """
        position - an array of shape [2,]
        orientation - float
        """
        self.position = np.array(position, dtype=np.float)
        self.orientation = orientation
        self.v = velocity
        self.name = name
        
        self.alive = True
        
    def turn(self, dtheta, dt=0.1):
        self.orientation += dtheta

    def move(self, dt=0.1):
        s_pos, s_or = self.position, self.orientation
        dp = np.array((np.cos(s_or) * self.v * dt, np.sin(s_or) * self.v * dt))
        self.position += dp

spaceship = Gobject((1.0,1.0), 0.5, velocity=1)
for a in range(10):
    spaceship.move()
    spaceship.turn(0.1)
    print(spaceship.position, spaceship.orientation)

[1.08775826 1.04794255] 0.6
[1.17029182 1.1044068 ] 0.7
[1.24677604 1.16882857] 0.7999999999999999
[1.31644671 1.24056418] 0.8999999999999999
[1.3786077  1.31889687] 0.9999999999999999
[1.43263793 1.40304397] 1.0999999999999999
[1.47799755 1.4921647 ] 1.2
[1.51423332 1.58536861] 1.3
[1.54098321 1.68172443] 1.4000000000000001
[1.55797992 1.7802694 ] 1.5000000000000002


Let's do this part first :)

3. Write a function or method to determine if the spaceship can hit its target.

In [7]:
def can_hit(spaceship, target, d, theta):
    """The spaceship can only hit when within a certain distance and angle from the asteroid."""
    s_pos, s_or = spaceship
    t_pos, t_or = target
    
    distance = np.sqrt(np.sum((s_pos - t_pos)**2))
    
    # it'll be easier if we convert to complex coordinates
    s_pos = complex(*s_pos)
    t_pos = complex(*t_pos)
    target_angle = np.angle(t_pos - s_pos)
    orientation_angle = np.angle(s_or)
    
    return distance <= d and np.abs(orientation_angle - target_angle) <= theta

def test_can_hit():
    # setup the targets
    spaceship = (np.array((0,0)), 0.0)
    target = (np.array((0,-1)), 1.0)
    
    d = 1 # has to be more than 1
    theta = 1.7 # has to be more than pi/2
    assert can_hit(spaceship, target, d, theta)
    
    d = 0.8
    assert not can_hit(spaceship, target, d, theta)
    
    d = 1 # has to be more than 1
    theta = 1.4 # has to be more than pi/2
    assert not can_hit(spaceship, target, d, theta)
    

test_can_hit()


2. Outline the code for the spaceship, asteroid, and missile.
    * Show any data structures and stubs for methods and/or functions that show the implementation of your design in part (a).
    * State any assumptions you make.
    * It is not necessary to make a functional game (in particular, do not worry about graphics or a game engine).

In [30]:

MAX_SHIP_SPEED = 4
MISSILE_SPEED = 8

class Spaceship(Gobject):
    """A Gobject that can be controlled"""
    def __init__(self, position, orientation, velocity=0.0, name="bob"):
        super(Spaceship, self).__init__(position, orientation, velocity, "spaceship_" + name)
        self.max_speed = MAX_SHIP_SPEED
    
    def turn(self, x):
        """Change the orientation by adding x"""
        self.orientation += x
        
    def accelerate(self, x):
        """Change the velocity by adding x"""
        self.v = max(0, min(self.v + x, self.max_speed))
        
    
class Asteroid(Gobject):
    def __init__(self, position, orientation, velocity=0.0, name="bob"):
        super(Asteroid, self).__init__(position, orientation, velocity, "asteroid_" + name)

        
class Missile(Gobject):
    """A Gobject that has different move strategies"""
    def __init__(self, position, orientation, velocity=0.0, name="bob"):
        super(Missile, self).__init__(position, orientation, velocity, "missile_" + name)
        
    def move(self):
        """Overrides the default move method by checking if it's exhausted its range"""
        super(Missile, self).move()
        
        
class Game:
    def __init__(self, objects):
        self.objects = []
        self.spaceship = None
        self.asteroid = None
        
        for o in objects:
            self.add(o)
        
        # hook up to user events
        print("User events are not supported!")
        self.can_run = True
        
        self.time = 0
    
    def add(self, o):
        """Add an object to the game"""
        if "spaceship" in o.name:
            self.spaceship = o
        elif "asteroid" in o.name:
            self.asteroid = o
        
        self.objects.append(o)
        
    def update(self):
        for g in self.objects:
            g.move()
        
        self.collision_test()
        self.render()
        self.time += 1
    
    def run(self):
        if not self.can_run:
            print("You need a front end to this game")
            return
        
        while self.spaceship.alive and self.time < 20:
            self.update()
            
        if not self.spaceship.alive:
            print("Game over!")
    
    def render(self):
        """Yes, no rendering, we just display the position of the asteroid"""
        print('Asteroid: {}'.format(self.asteroid.position) if self.asteroid.alive else "", \
              'You: {}'.format(self.spaceship.position) if self.spaceship.alive else "")
        missiles = [o for o in self.objects if "missile" in o.name]
        for o in missiles:
            print('Missile:', o.position)
        
    def collision_test(self):
        """Removes collided objects"""
        for a in self.objects:
            if not a.alive:
                continue
            for b in self.objects:
                if not b.alive:
                    continue
                
                a_data = (a.position, a.orientation)
                b_data = (b.position, b.orientation)
                
                # missile-asteroid collisions
                # the missile has to pointed AT the asteroid
                distance = 2
                PI = 3.1415926535
                theta = PI/10
                if "missile" in a.name and "asteroid" in b.name and can_hit(a_data, b_data, distance, theta):
                    a.alive = False
                    b.alive = False
                    
                # asteroid-ship collisions
                if "asteroid" in a.name and "spaceship" in b.name and can_hit(a_data, b_data, distance, theta):
                    a.alive = False
                    b.alive = False
                            
        # remove all the dead objects
        self.objects = [x for x in self.objects if x.alive]
    
    def fire(self):
        """Fires a missile at MISSILE_SPEED in the current spaceship direction
        
        Allow the player to fire as many missiles as they want
        """
        self.objects.append(Missile(self.spaceship.position, self.spaceship.orientation, velocity=MISSILE_SPEED))
    
    def user_input(self, c):
        """Called externally!
        
        Assume that spaceship accelerate methods are safe when
        used asynchronously!
        """
        if c == "ACCELERATE":
            self.spaceship.accelerate(0.1)
        elif c == "DECELERATE":
            self.spaceship.accelerate(-0.1)
        elif c == "TURN_LEFT":
            self.spaceship.turn(0.1)
        elif c == "TURN_RIGHT":
            self.spaceship.turn(-0.1)
        elif c == "FIRE":
            self.fire()


Let's play the game!

In [31]:
"""
         * -- incoming asteroid!!   
              
       >            
       /                
    (you) 
"""

g = Game([
    # add a spaceship, which the asteroid will hit around 3 timesteps
    Spaceship((-0.15145383, 0.45896281), 0),
    Asteroid((0,0.2), 2.1, velocity=1.0)
])
g.run()

User events are not supported!
Asteroid: [-0.05048461  0.28632094] You: [-0.15145383  0.45896281]
Asteroid: [-0.10096922  0.37264187] You: [-0.15145383  0.45896281]
 
Game over!


In [34]:
"""
          (missile going right)
              |
       >      *          *  
       /                  \
    (you)                  (asteroid)
"""
g = Game([
    Spaceship((0, 0), 0),
    Asteroid((10,0), 2.1),
])
g.user_input("FIRE")
g.run()

User events are not supported!
Asteroid: [10.  0.] You: [0. 0.]
Missile: [0.8 0. ]
Asteroid: [10.  0.] You: [0. 0.]
Missile: [1.6 0. ]
Asteroid: [10.  0.] You: [0. 0.]
Missile: [2.4 0. ]
Asteroid: [10.  0.] You: [0. 0.]
Missile: [3.2 0. ]
Asteroid: [10.  0.] You: [0. 0.]
Missile: [4. 0.]
Asteroid: [10.  0.] You: [0. 0.]
Missile: [4.8 0. ]
Asteroid: [10.  0.] You: [0. 0.]
Missile: [5.6 0. ]
Asteroid: [10.  0.] You: [0. 0.]
Missile: [6.4 0. ]
Asteroid: [10.  0.] You: [0. 0.]
Missile: [7.2 0. ]
Asteroid: [10.  0.] You: [0. 0.]
Missile: [8. 0.]
 You: [0. 0.]
 You: [0. 0.]
 You: [0. 0.]
 You: [0. 0.]
 You: [0. 0.]
 You: [0. 0.]
 You: [0. 0.]
 You: [0. 0.]
 You: [0. 0.]
 You: [0. 0.]


# Question 3

Suppose we have a fleet of 10 identical Autonomous Vehicles with the same sensor suite and storage capabilities.
1. How much data would this fleet collect over a week?
    * State your assumptions and show your calculations.
    * There are many possible exact values for this question. We are looking at your process, assumptions, and logic.

## Assumptions

### Hardware on the car

Let's try to get an estimate on how much data a single car can collect in an hour of driving.

**Cameras** - Assume we have 6 cameras (front main 1, front main 2, side 1, side 2, back 1, back 2).
* Resolution - Each camera might output a 500x1000 image, at 25Hz.

**Lidars** - Assume we don't have LIDAR

**Control sensors** - Assume that we can record things about a human demonstrator as well:
* Pressure on the gas pedal - float, 25Hz
* Pressure on the brake pedal - float, 25Hz
* Steering wheel angle - float, 25Hz
* Whether the signal lights are activated - tuple, 25Hz

**System sensors** - Assume that we also have metrics on the car:
* Temperature of the engine - float, 25Hz
* Gas left? - float, 25hz

I don't think we have any sensors that come close to the cameras :) in terms of raw data

### How the cars are being used

Now, even with this hardware, the cars might be driven in different environments, and used in different ways. Assume that as long as a car is ON, it's collecting the same rate of data no matter its use case.

**Uptime** - 10 hours / day, 7 days / week



In [11]:
resolution = 500 * 1000
hz = 25
n_cameras = 6
n_cars = 10

camera_per_second = resolution * hz
camera_per_hour = camera_per_second * 3600
print(camera_per_hour / 1e9, 'Gb per camera/hour')
# Why is a single camera capturing so much when a movie can be compressed to 2gb/hour?

camera_per_week = n_cameras * camera_per_hour * 7 * 10
print(camera_per_week / 1e12, 'Tb per car/week')

print(camera_per_week * n_cars / 1e12, 'Tb per fleet/week (10 cars)')

45.0 Gb per camera/hour
18.9 Tb per car/week
189.0 Tb per fleet/week (10 cars)


In [12]:
# with av1 compression?
camera_per_hour = 3e9
print(camera_per_hour / 1e9, 'Gb per camera/hour')

camera_per_week = n_cameras * camera_per_hour * 7 * 10
print(camera_per_week / 1e12, 'Tb per car/week')

print(camera_per_week * n_cars / 1e12, 'Tb per fleet/week (10 cars)')

3.0 Gb per camera/hour
1.26 Tb per car/week
12.6 Tb per fleet/week (10 cars)


# Question 4

Write a program which counts how many possible games of TicTacToe consist of a certain number of moves.

Output format example (these numbers are incorrect):
```
5 78492
6 289
7 48901
8 8172
9 231
```

How do we do this efficiently? 

In [13]:
# basic representation

# the board 
# [ 0 1 2 ]
# [ 3 4 5 ]
# [ 6 7 8 ]

wins = [
    # horizontal
    [0,1,2],
    [3,4,5],
    [6,7,8],
    
    # vertical
    [0,3,6],
    [1,4,7],
    [2,5,8],
    
    # diagonal
    [0,4,8],
    [2,4,6]
]

# the idea is to only check the wins that occur for each square
# NOTE: this didn't save us *that* much time!
possible_wins = {
    square:[w for w in wins if square in w] for square in range(9)
}

possible_wins

{0: [[0, 1, 2], [0, 3, 6], [0, 4, 8]],
 1: [[0, 1, 2], [1, 4, 7]],
 2: [[0, 1, 2], [2, 5, 8], [2, 4, 6]],
 3: [[3, 4, 5], [0, 3, 6]],
 4: [[3, 4, 5], [1, 4, 7], [0, 4, 8], [2, 4, 6]],
 5: [[3, 4, 5], [2, 5, 8]],
 6: [[6, 7, 8], [0, 3, 6], [2, 4, 6]],
 7: [[6, 7, 8], [1, 4, 7]],
 8: [[6, 7, 8], [2, 5, 8], [0, 4, 8]]}

In [14]:
def checkWin(board, pattern):
    return board[pattern[0]] == board[pattern[1]] and board[pattern[0]] == board[pattern[2]]

def pprint(board):
    print(board[:3])
    print(board[3:6])
    print(board[6:])
    print('--------------')
    
def calculateWinNumber(game, disp=False):
    board = [' '] * 9
    mark = 'x'
    for moveNumber, move in enumerate(game):
        move = int(move)
        board[move] = mark
        if disp:
            pprint(board)
        mark = 'o' if mark == 'x' else 'x'
        # check for win
        for pattern in possible_wins[move]:
            if checkWin(board, pattern):
                return moveNumber + 1
    
    # tie? / no one wins
    return -1

print(calculateWinNumber('40236', True))

[' ', ' ', ' ']
[' ', 'x', ' ']
[' ', ' ', ' ']
--------------
['o', ' ', ' ']
[' ', 'x', ' ']
[' ', ' ', ' ']
--------------
['o', ' ', 'x']
[' ', 'x', ' ']
[' ', ' ', ' ']
--------------
['o', ' ', 'x']
['o', 'x', ' ']
[' ', ' ', ' ']
--------------
['o', ' ', 'x']
['o', 'x', ' ']
['x', ' ', ' ']
--------------
5


In [15]:
import itertools
import random

def bruteForce(LOG=False, SAMPLE=False):
    # a game is represented by 0-8
    # ex: 012345678
    
    counters = [0 for i in range(12)] # the last one is -1

    all_permutations = itertools.permutations('012345678')
    counter = 0
    for game in all_permutations:
        
        # log
        counter += 1
        if LOG and counter % 1000 == 0:
            print(counter // 1000, counters)
        
        # sample games to see if it's calculating correctly
        if SAMPLE and random.random() < 0.001:
            print(calculateWinNumber(game, True))
            
        counters[calculateWinNumber(game)] += 1
    
    # format it in the right format
    for i in [5,6,7,8,9]:
        k = counters[i]
        print("{} {}".format(i, k))
        
    return counters
        

bruteForce()

5 34560
6 31968
7 95904
8 72576
9 81792


[0, 0, 0, 0, 0, 34560, 31968, 95904, 72576, 81792, 0, 46080]

# Question 5


A rival AI company, Tesmo, has made two autonomous vehicles. Fortunately they aren’t very good. They only have two cars with the same program, which consists of at most 10 instructions. Each instruction is one of:

* 👈 Drive backwards one car length
* 👉 Drive forwards one car length
* 🙈 Skip the next instruction if there’s a crater under the car
* An integer 0-9 - Jump to the instruction corresponding to this index, e.g. “1” jumps to the second instruction in the program

**Write a program** that ensures the vehicles will crash into each other (clients want the strangest things).

a. The program should be identical for both cars.

b. They should crash regardless of where they start.

c. Assume that executing each instruction takes exactly the same amount of
time and that both cars start execution at the same time.

## Actual solution

The key insight is that limiting to 10 instructions dramatically simplifies the space of things that the cars can do - nothing too complicated. In particular, the car that goes forward without hitting a crater will have no idea when to turn around! To deal with this, we put all the intelligence in the car that hits the second crater.

We start with two cars with the following program:

* go FORWARD at half-speed until you hit a crater (one of the cars, w.l.o.g. Sammy, will do do this)
* go FORWARD at full-speed, and catch up with the other car who's still going at half-speed

In [18]:
FORWARD = 'f'
BACK = 'b'
SKIP = 's'
JUMP = list(range(10))

# proceed at half-speed, then at full speed
program = [
    FORWARD, FORWARD, BACK, SKIP, 0, FORWARD, SKIP, 5
]

# in terms of time, it takes roughly 2*5*d timesteps for them to collide (since the instruction loop has 5 instructions)

## Funny story

I actually started making a simulator, because I thought it'd be fun + necessary. Turns out it wasn't necessary (thinking goes a long way!) but it was fun :)

When making this program, I thought of two main approaches:

1. Make the cars execute independently, and then let the world do stuff like collisions
    * Pros: You can reuse the interpreter loop, this has a nice object-oriented feel
    * Cons: Might need lots of logic for external stuff like collisions, more difficult to add custom behavior in response to world events
2. Make the world responsible for checking collisions *AND* for updating the cars according to the program
    * Pros: might be neater, bc you don't need to communicate as much (the world has all the state)
    * Cons: Since the cars will end up with different program counters, you might be duplicating logic somehow?

Ultimately I chose (1), because it seemed easier to test with one car!

In [19]:
def crater(x):
    return x == 10

class Car:
    """This is a basic interpreter loop...
    
    The car can update its own position based on the program!
    
    """
    def __init__(self, name = "CAR", pos = 10):
        self.initial_pos = pos
        self.name = name
        self.can_execute = True
    
    def alert(self, msg):
        print("{} {}".format(self.name, msg))
        
    def msg(self, msg):
        if self.verbose:
            print("{} {}".format(self.name, msg))
        
    
    def execute(self, program, crater, verbose=True, LIMIT=40):
        """Executes the program on the car!
        
        Inputs:
        program - a list of instructions like [FORWARD, BACK, SKIP, 1]
        crater - a function that tells the car where the world's craters are
        
        Outputs:
        Yields (time, program counter, instruction, position) tuples
        
        Also prints messages for halting and skipping instructions
        """
        self.verbose = verbose
        position = self.initial_pos
        
        counter = 0
        time = 0
        skip = False
        while counter < len(program) and self.can_execute:
  
            if skip:
                skip = False
                if crater(position):
                    self.alert("Skipping the current instruction {} {}".format(counter, program[counter]))
                    counter += 1
                    continue
            
            time += 1
            if time > LIMIT:
                self.alert("OUT OF GAS!")
                break
                
            instruction = program[counter]
            self.msg("t={} {} {}\t {}".format(time, counter, instruction, position))
            yield (time, counter, instruction, position)

            if instruction == FORWARD:
                position += 1
            elif instruction == BACK:
                position -= 1
            elif instruction == SKIP:
                skip = True
            elif instruction in JUMP:
                counter = instruction
                continue
            
            counter += 1
        
    def crash(self):
        self.can_execute = False
        self.alert("BOOOOM!")


In [20]:
c = Car("Bog the Builder")
steps = c.execute([
    FORWARD, BACK, FORWARD, BACK, SKIP, FORWARD, BACK, SKIP
], crater)
list(steps)

Bog the Builder t=1 0 f	 10
Bog the Builder t=2 1 b	 11
Bog the Builder t=3 2 f	 10
Bog the Builder t=4 3 b	 11
Bog the Builder t=5 4 s	 10
Bog the Builder Skipping the current instruction 5 f
Bog the Builder t=6 6 b	 10
Bog the Builder t=7 7 s	 9


[(1, 0, 'f', 10),
 (2, 1, 'b', 11),
 (3, 2, 'f', 10),
 (4, 3, 'b', 11),
 (5, 4, 's', 10),
 (6, 6, 'b', 10),
 (7, 7, 's', 9)]

## Dealing with two cars

In [21]:
def run_cars(positions, program, LIMIT=30):
    """Runs the same program on both cars. Checks if they collide!
    
    `run_cars` prints out the positions of each car at every timestep
    
    Inputs:
    - positions: a tuple of 2 positions
    - program:
    
    Returns nothing
    """
    car1 = Car("Car1", positions[0])
    car2 = Car("Car2", positions[1])
    crater = lambda a: a in positions

    A,B = car1.execute(program, crater, False, LIMIT), car2.execute(program, crater, False, LIMIT)
    def check(a,b):
        posA = a[-1]
        posB = b[-1]
        print('time {} : {} {}'.format(max(a[0], b[0]), posA, posB))
        if posA == posB:
            car1.crash()
            car2.crash()
            
    for a,b in zip(A,B):
        check(a,b)

    # when one car stops, keep going until the other car stops
    for a in A:
        check(a,b)
    for b in B:
        check(a,b)

# if we hit a crater, we stop going forward
program = [
    FORWARD, SKIP, 0
]
run_cars([10,11], program, 10)

time 1 : 10 11
time 2 : 11 12
Car1 Skipping the current instruction 2 0
time 3 : 11 12
time 4 : 11 12
time 5 : 11 13
time 6 : 11 13
time 7 : 11 13
time 8 : 11 14
time 9 : 11 14
time 10 : 11 14
Car2 OUT OF GAS!


## Finally, our original solution:

In [22]:
# proceed at half-speed, then at full speed
program = [
    FORWARD, FORWARD, BACK, SKIP, 0, FORWARD, SKIP, 5
]

# in terms of time, it takes roughly 2*5*d timesteps for them to collide (since the instruction loop has 5 instructions)
run_cars([10,14], program, 110)

time 1 : 10 14
time 2 : 11 15
time 3 : 12 16
time 4 : 11 15
time 5 : 11 15
time 6 : 11 15
time 7 : 12 16
time 8 : 13 17
time 9 : 12 16
time 10 : 12 16
time 11 : 12 16
time 12 : 13 17
time 13 : 14 18
time 14 : 13 17
time 15 : 13 17
time 16 : 13 17
time 17 : 14 18
time 18 : 15 19
time 19 : 14 18
Car1 Skipping the current instruction 4 0
time 20 : 14 18
time 21 : 15 18
time 22 : 15 19
time 23 : 15 20
time 24 : 16 19
time 25 : 16 19
time 26 : 16 19
time 27 : 17 20
time 28 : 17 21
time 29 : 17 20
time 30 : 18 20
time 31 : 18 20
time 32 : 18 21
time 33 : 19 22
time 34 : 19 21
time 35 : 19 21
time 36 : 20 21
time 37 : 20 22
time 38 : 20 23
time 39 : 21 22
time 40 : 21 22
time 41 : 21 22
time 42 : 22 23
time 43 : 22 24
time 44 : 22 23
time 45 : 23 23
Car1 BOOOOM!
Car2 BOOOOM!
